In [163]:
import random
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial

class Server:
    def __init__(self, x_val, prime):
        self.x_val = x_val
        self.prime = prime
        self.shares = [0, 0, 0]
    
    def add_input(self, new_shares):
        for i in range(len(self.shares)):
            self.shares[i] += new_shares[i]
        
    def verify_input(self, new_shares, other_x, r):
        transform_factor = -other_x / (self.x_val - other_x)
        transformed_shares = [transform_factor * share for share in new_shares]
        self.H = create_h(transformed_shares, self.prime)
        return self.H.calculate(r)
    
    def divide(self, a, b):
        return self.multiply(a, pow(b, -1, self.prime))
    
    def multiply(self, a, b):
        return (a * b) % self.prime

class H_Function:
    def __init__(self, f, g, l, p):
        self.f = f
        self.g = g
        self.length = l
        self.prime = p
    
    def calculate(self, x):
        x_vals = [x**i for i in range(self.length)]
        f_total = sum([x_vals[i] * self.f[i] for i in range(self.length)])
        g_total = sum([x_vals[i] * self.g[i] for i in range(self.length)])
        return f_total % self.prime, g_total % self.prime

def generate_secret_shares(secret_val, prime):
    rand_val = random.randint(2, 5)
    diff = rand_val - secret_val
    shares = [rand_val, (rand_val + diff) % prime, (rand_val + 2*diff) % prime]
    return shares    

def create_h(secret_vals, prime):
    f = lagrange([i for i in range(1, len(secret_vals) + 1)], secret_vals)
    g = lagrange([i for i in range(1, len(secret_vals) + 1)], [s - 1 for s in secret_vals])
    
    f_coeff = Polynomial(f.coef[::-1]).coef
    g_coeff = Polynomial(g.coef[::-1]).coef
    return H_Function(f_coeff, g_coeff, len(f_coeff), prime)

def combine_shares(server_1, server_2, prime):
    return ((server_1[0] + server_2[0]) * (server_1[1] + server_2[1] + 1)) % prime

In [215]:
prime = 29
A = Server(1, prime)
B = Server(2, prime)
C = Server(3, prime)
server_dict = {'A': (A, 1), 'B': (B, 2), 'C': (C, 3)}

secrets = [random.randint(0, 1) for i in range(3)]
secret_shares = [generate_secret_shares(s, prime) for s in secrets]
new_shares = list(zip(secret_shares[0], secret_shares[1], secret_shares[2]))
H = create_h(secrets, prime)
print(f'Secret values: {secrets}')
print(f'Shares for each secret value: {secret_shares}')

Secret values: [1, 1, 0]
Shares for each secret value: [[2, 3, 4], [5, 9, 13], [3, 6, 9]]


In [216]:
SERVER_1 = 'A'
SERVER_2 = 'B'
r = 8

In [217]:
# computations in this cell to be done in a secure multiparty manner

# first server performing verification step
server, x_val = server_dict[SERVER_1]
server1_shares = server.verify_input(new_shares[x_val-1], server_dict[SERVER_2][1], r)

# second server performing verification step
server, x_val = server_dict[SERVER_2]
server2_shares = server.verify_input(new_shares[x_val-1], server_dict[SERVER_1][1], r)

print(f'Server 1 shares\nf({r}) = {server1_shares[0]}\ng({r}) = {server1_shares[1]}\n')
print(f'Server 2 shares\nf({r}) = {server2_shares[0]}\ng({r}) = {server2_shares[1]}\n')

combined = combine_shares(server1_shares, server2_shares, prime)
print(f'Combined result: f({r})g({r}) = {combined}')

Server 1 shares
f(8) = 10.0
g(8) = 9.0

Server 2 shares
f(8) = 28.0
g(8) = 27.0

Combined result: f(8)g(8) = 14.0


In [218]:
x, y = H.calculate(r)
print(f'True H Function Value: H({r}) = {(x * y) % prime}')

True H Function Value: H(8) = 14.0
